<a href="https://colab.research.google.com/github/vnmrsharma/CANIS/blob/main/Canis_hackathon_notebook_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CANIS hackathon notebook

## Import libraries and modules

In [1]:
import pandas as pd

# !pip install -q tensorflow-model-optimization

import tempfile
import os
import time

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

from tensorflow import keras

# import pillow
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pysentimiento
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")
emotion_analyzer = create_analyzer(task="emotion", lang="en")
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="en")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
! pip install geotext
from geotext import GeoText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Load data

In [6]:
combined_df = pd.read_csv("/content/drive/MyDrive/Canis_hackathon/Cleaned.csv")
display(combined_df)

,text,T_F_Score
0,"['head', 'conservative', 'republican', 'factio...",1
1,"['transgender', 'people', 'allowed', 'first', ...",1
2,"['special', 'counsel', 'investigation', 'link'...",1
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1
4,"['president', 'donald', 'trump', 'called', 'u'...",1
...,...,...
68547,"['apparently', 'new', 'kyiv', 'government', 'h...",0
68548,"['usa', 'want', 'divide', 'syria', 'great', 'b...",0
68549,"['ukrainian', 'coup', 'detat', 'cost', 'u', 'n...",0
68550,"['european', 'parliament', 'falsifies', 'histo...",0


In [7]:
combined_df["text_str"] = combined_df["text"].apply(lambda x: x.title().replace("[",""))
combined_df["text_str"] = combined_df["text_str"].apply(lambda x: x.replace("]",""))
combined_df["text_str"] = combined_df["text_str"].apply(lambda x: x.replace("'",""))
combined_df["text_str"] = combined_df["text_str"].apply(lambda x: x.replace(",",""))

In [8]:
combined_df = combined_df.iloc[:10] # just get the top 1000 rows
display(combined_df)

,text,T_F_Score,text_str
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...


In [9]:
# true_df = pd.read_csv("/content/drive/MyDrive/Canis_hackathon/DataSet_Misinfo_TRUE.csv")
# # fake_df = pd.read_csv("/content/drive/MyDrive/Canis_hackathon/DataSet_Misinfo_FAKE.csv")

# display(true_df)
# # display(fake_df)

## Data cleaning

In [10]:
# true_df = true_df.drop('Unnamed: 0',axis=1)
# true_df["label"] = 1

# display(true_df)

In [11]:
# fake_df = fake_df.drop('Unnamed: 0',axis=1)
# fake_df["label"] = 0

# display(fake_df)

## Data segmentation/classification

### Word clouds


word clouds for the true dataset

In [12]:
# comment_words = ''
# stopwords = set(STOPWORDS)
 
# # iterate through the csv file
# for val in combined_df.text:
     
#     # typecaste each val to string
#     val = str(val)
 
#     # split the value
#     tokens = val.split()
     
#     # Converts each token into lowercase
#     for i in range(len(tokens)):
#         tokens[i] = tokens[i].lower()
     
#     comment_words += " ".join(tokens)+" "
 
# wordcloud = WordCloud(width = 1024, height = 800,
#                 background_color ='white',
#                 stopwords = stopwords,
#                 min_font_size = 10).generate(comment_words)
 
# # plot the WordCloud image                      
# plt.figure(figsize = (8, 8), facecolor = None)
# plt.imshow(wordcloud)
# plt.axis("off")
# plt.tight_layout(pad = 0)
 
# plt.show()

## Sentiment, emotion and hate speech analysis

### Data preprocessing for sentiment analysis

In [13]:
# preprocess_tweet("📢 @Helooooooooooooo this is #BreakingNews")

In [14]:
# x = preprocess_tweet("I like Elon Musk soooooooooooo muchhhhhhhhhhhhhh!!!!!!!!!!!")
# print(x)

In [15]:
# true_df["preprocessed"] = combined_df["text"].apply(lambda x: preprocess_tweet(str(x)))

# display(true_df)

## Sentiment analysis column

In [16]:
x = sentiment_analyzer.predict("I like Elon Musk")
print(x)

AnalyzerOutput(output=POS, probas={POS: 0.971, NEU: 0.028, NEG: 0.002})


In [17]:
combined_df["sentiment_analysis"] = combined_df["text_str"].apply(lambda x: sentiment_analyzer.predict(x).output)
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU


## Emotion analysis

In [18]:
combined_df["emotion_analysis"] = combined_df["text_str"].apply(lambda x: emotion_analyzer.predict(x).output)
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis,emotion_analysis
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU,others
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU,others
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU,others
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU,disgust
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU,others
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU,others
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU,others
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG,disgust
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS,joy
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU,others


## Hate speech analysis

In [19]:
combined_df["hate_speech_analysis"] = combined_df["text_str"].apply(lambda x: hate_speech_analyzer.predict(x).output)
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis,emotion_analysis,hate_speech_analysis
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU,others,[]
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU,others,[]
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU,others,[]
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU,disgust,[]
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU,others,[]
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU,others,[]
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU,others,[]
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG,disgust,[]
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS,joy,[]
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU,others,[]


##Location information

In [20]:
# def get_country(text_list):

#   places = GeoText(raw)
#   cities = list(places.cities)
#   country_name_to_check_list =  [str(country.name).lower() for country in pycountry.countries]

#   list_for_search = country_name_to_check_list.append(cities)
#   # print(country_name_to_check_list)

#   for word in text_list:

#     word = str(word).lower()

#     if word in list_for_search:
#       return word

#   return None

def get_country_new(text_list):

  places = GeoText(text_list)
  countries = list(places.countries)
  return countries

def get_cities_new(text_list):

  places = GeoText(text_list)
  cities = list(places.cities)
  return cities 

def get_country_mentions_new(text_list):
  country_mentions = GeoText(text_list).country_mentions
  return dict(country_mentions)

def get_cities_mentions_new(text_list):
  cities_mentions = GeoText(text_list).cities_mentions
  return cities_mentions

In [21]:
combined_df["country"] = combined_df["text_str"].apply(lambda x: get_country_new(x))
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis,emotion_analysis,hate_speech_analysis,country
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU,others,[],[]
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU,others,[],[]
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU,others,[],[]
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU,disgust,[],[]
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU,others,[],[]
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU,others,[],[]
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU,others,[],[]
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG,disgust,[],[]
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS,joy,[],[]
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU,others,[],[]


In [22]:
combined_df["city"] = combined_df["text_str"].apply(lambda x: get_cities_new(x))
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis,emotion_analysis,hate_speech_analysis,country,city
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU,others,[],[],[]
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU,others,[],[],[]
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU,others,[],[],[New York]
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU,disgust,[],[],[New York]
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU,others,[],[],[]
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU,others,[],[],[]
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU,others,[],[],[New York]
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG,disgust,[],[],[]
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS,joy,[],[],[]
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU,others,[],[],[]


In [23]:
combined_df["country_mentions"] = combined_df["text_str"].apply(lambda x: get_country_mentions_new(x))
display(combined_df)

,text,T_F_Score,text_str,sentiment_analysis,emotion_analysis,hate_speech_analysis,country,city,country_mentions
0,"['head', 'conservative', 'republican', 'factio...",1,Head Conservative Republican Faction U Congres...,NEU,others,[],[],[],{}
1,"['transgender', 'people', 'allowed', 'first', ...",1,Transgender People Allowed First Time Enlist U...,NEU,others,[],[],[],{}
2,"['special', 'counsel', 'investigation', 'link'...",1,Special Counsel Investigation Link Russia Pres...,NEU,others,[],[],[New York],{'US': 1}
3,"['trump', 'campaign', 'adviser', 'george', 'pa...",1,Trump Campaign Adviser George Papadopoulos Tol...,NEU,disgust,[],[],[New York],{'US': 1}
4,"['president', 'donald', 'trump', 'called', 'u'...",1,President Donald Trump Called U Postal Service...,NEU,others,[],[],[],{}
5,"['white', 'house', 'said', 'friday', 'set', 'k...",1,White House Said Friday Set Kick Talk Next Wee...,NEU,others,[],[],[],{}
6,"['president', 'donald', 'trump', 'said', 'thur...",1,President Donald Trump Said Thursday Belief Fa...,NEU,others,[],[],[New York],{'US': 1}
7,"['fake', 'news', 'love', 'talk', 'socalled', '...",1,Fake News Love Talk Socalled Low Approval Rati...,NEG,disgust,[],[],[],{}
8,"['together', 'making', 'america', 'great', 'bi...",1,Together Making America Great Bitlylnpkaq Est,POS,joy,[],[],[],{}
9,"['alabama', 'secretary', 'state', 'john', 'mer...",1,Alabama Secretary State John Merrill Said Cert...,NEU,others,[],[],[],{}


## Save file

In [24]:
combined_df.to_csv("/content/drive/MyDrive/Canis_hackathon/Cleaned_1.csv")

## Data model implementation

## Data model evaluation

## Data model infographics compilation